<h1>Download One Specific File</h1>

In [6]:
import os
import requests

In [7]:
# download xlsx file, ppp-data-dictionary

dictionary_url = "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/aab8e9f9-36d1-42e1-b3ba-e59c79f1d7f0/download/ppp-data-dictionary.xlsx"
resp = requests.get(dictionary_url)
with open('ppp-data-dictionary.xlsx', 'wb') as output:
    output.write(resp.content)

In [8]:
# download one csv file, 

CSV_URL = 'https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/7371d1ca-57e8-4efb-9993-780cea937449/download/public_up_to_150k_12_230331.csv'
with open(os.path.split(CSV_URL)[1], 'wb') as f, \
        requests.get(CSV_URL, stream=True) as r:
    for line in r.iter_lines():
        f.write(line+'\n'.encode())


<h1>Download All the Files of PPP Loan</h1>

In [32]:
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient
import os
import requests
from bs4 import BeautifulSoup
import re
import boto3
import glob
import pandas as pd

In [14]:

url = "https://data.sba.gov/dataset/ppp-foia"
# Download the webpage content
response = requests.get(url)
page_content = response.text
print(f"Downloaded {csv_filename} to {destination_file}.")
print("All CSV files have been downloaded.")

Downloaded public_150k_plus_230630.csv to data/ppploan\public_150k_plus_230630.csv.
All CSV files have been downloaded.


In [37]:
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")
# Find all anchor tags containing CSV file links
csv_links = soup.find_all("a", href=re.compile(".*\.csv"))
# Define the destination folder for the downloaded CSV files
destination_folder = "./data/ppploan"

In [40]:
# Download each CSV file to the specified destination folder
for link in csv_links:
    csv_url = link["href"]
    csv_filename = os.path.basename(csv_url)
    destination_file = os.path.join(destination_folder, csv_filename)
    print(f"Downloading {csv_url}...")
    # Download the CSV file
    with requests.get(csv_url, stream=True) as r:
        r.raise_for_status()
        with open(destination_file, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

<h1> Upload All the PPP Loan Files to Azure</h1>

In [52]:
folders = ["data/ppploan"]

# Azure Blob Storage Configuration
your_azure_connection_string = 'DefaultEndpointsProtocol=https;AccountName=sapppcis4400dev;AccountKey=87qq/ubn/GtI/KxMzWUfxVvEu5hnWhOSa9FSBmYq02i1HVKDOKD13Yc28Vh512tBDqR2GsThKf7c+AStZw5HOg==;EndpointSuffix=core.windows.net'
azure_connection_string = your_azure_connection_string
your_azure_container_name = 'sapppcis4400dev'
azure_container_name = your_azure_container_name


In [46]:
# Upload files to Azure Blob Storage

def upload_to_azure(source_file, destination_path):
    blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
    container_client = blob_service_client.get_container_client(azure_container_name)
    blob_client = container_client.get_blob_client(destination_path)
    with open(source_file, "rb") as data:
        blob_client.upload_blob(data)


In [53]:
# Iterate through each folder, and upload the CSV files to S3, GCS, and Azure

for folder in folders:
    csv_files = glob.glob(os.path.join(folder, "*.csv"))
    for csv_file in csv_files:
        destination_path = os.path.join("uploaded", os.path.basename(csv_file))
        # Upload to Azure Blob Storage
        upload_to_azure(csv_file, destination_path)
        print(f"Uploaded {csv_file} to S3, GCS, and Azure.")

ResourceNotFoundError: The specified container does not exist.
RequestId:9363ad50-001e-0049-509c-afa215000000
Time:2023-07-05T23:58:32.1809687Z
ErrorCode:ContainerNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerNotFound</Code><Message>The specified container does not exist.
RequestId:9363ad50-001e-0049-509c-afa215000000
Time:2023-07-05T23:58:32.1809687Z</Message></Error>